Desenvolvendo uma tabela que contenha os seguintes dados: 
- o identificador do discente; 
- o sexo do discente;
- o ano em que o discente ingressou na instituição;
- o ano e período da última matrícula realizada pelo discente;
- total de sementres cursados pelo discente;
- um calcúlo de semestres cursados dividido pelo o número de semestre ideal do curso;
- a carga horária cumprida pelo discente;
- a carga horária cumprida pelo discente dividida pela carga horaria ideal;
- tempo relativo que o discente evadiu ou concluiu o curso;
- status atual do discente;
- número total de faltas de cada discente na disciplina que ele cursou;
- nota de cada discente nas disciplinas que ele cursou;
- média final de cada discente nas disciplinas que ele cursou;
- quantidade de vezes que o discente se matriculou nas disciplinas;
- quantidade de vezes que o discente se matriculou em cada disciplina;
- quantidade de vezes que o discente foi aprovado em cada disciplina;
- quantidade de vezes que o discente foi reprovado em cada disciplina.

# Extraindo Dados
___

Importando o Pandas e o Csv.

In [396]:
import pandas as pd
import csv

Leitura do arquivo em csv e carregamento desses dados em um DataFrame usando o ponto e vírgula como separador.

In [397]:
df_dados = pd.read_csv('dataframe-bsi-2009-2022.csv', sep=';')

Listando as colunas do dataframe.

In [398]:
df_dados.columns

Index(['discente', 'unidade', 'nota', 'media_final', 'numero_total_faltas',
       'descricao', 'ano', 'id_componente', 'nome', 'ch_total', 'sexo',
       'ano_nascimento', 'ano_ingresso', 'status'],
      dtype='object')

In [399]:
# Verificar os valores faltosos na coluna 'media_final' com base na condição da coluna 'descricao'
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'media_final'] = -2

In [400]:
# Verificar os valores faltosos na coluna 'media_final' com base na condição da coluna 'descricao'
condicao_excluida = df_dados['descricao'] == 'EXCLUIDA'
df_dados.loc[condicao_excluida, 'media_final'] = -3

In [401]:
# Verificar os valores faltosos na coluna 'media_final' com base na condição da coluna 'descricao'
condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'media_final'] = -5

In [402]:
# Verificar os valores faltosos na coluna 'media_final' com base na condição da coluna 'descricao'
condicao_trancado = df_dados['descricao'] == 'INDEFERIDO'
df_dados.loc[condicao_trancado, 'media_final'] = -4

In [403]:
# Verificar os valores faltosos na coluna 'media_final' com base na condição da coluna 'descricao'
condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'media_final'] = -8

# Filtros

Fazendo um recorte da nossa análise, vamos começar por disciplinas obrigatórias do Bacharelado em Sistemas da Informação (BSI):

In [404]:
lista_obrigatórias = [
                'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                'INTRODUÇÃO À INFORMÁTICA',
                'FUNDAMENTOS DE MATEMÁTICA',
                'LÓGICA',
                'TEORIA GERAL DA ADMINISTRAÇÃO',
                'PROGRAMAÇÃO',
                'CÁLCULO DIFERENCIAL E INTEGRAL',
                'TEORIA GERAL DOS SISTEMAS',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                'ESTRUTURA DE DADOS',
                'ÁLGEBRA LINEAR',
                'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                'PROGRAMAÇÃO WEB',
                'ARQUITETURA DE COMPUTADORES',
                'PROBABILIDADE E ESTATÍSTICA',
                'BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE I',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                'SISTEMAS OPERACIONAIS',
                'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE II',
                'REDES DE COMPUTADORES',
                'EMPREENDEDORISMO EM INFORMÁTICA',
                'GESTÃO DE PROJETO DE SOFTWARE',
                'PROGRAMAÇÃO VISUAL',
                'MATEMÁTICA FINANCEIRA',
                'SISTEMAS DE APOIO À DECISÃO',
                'ÉTICA',
                ]
condição_nome = f"nome in {lista_obrigatórias}"
df_dados_filtrado = df_dados.query(condição_nome)

Listando os dados NaN.

In [405]:
df_dados_filtrado.isnull().sum()

discente                  0
unidade                2058
nota                   3979
media_final               0
numero_total_faltas    2340
descricao                 0
ano                       0
id_componente             0
nome                      0
ch_total                  0
sexo                      0
ano_nascimento            0
ano_ingresso              0
status                    0
dtype: int64

Preenchendo os dados NaN da coluna **unidade** com 1.

In [406]:
#df_dados_filtrado.loc[:, 'unidade'] = df_dados_filtrado['unidade'].fillna(1)

Filtrar os dados onde a coluna **unidade** foi preenchida com 1.

In [407]:
#df_dados_filtrado = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Renomeando dados da coluna descrição.

In [408]:
'''df_dados_filtrado = df_dados_filtrado.replace({'descricao':
               {'APROVADO POR NOTA':'APROVADO',
                'REPROVADO POR FALTAS':'REPROVADO',
                'REPROVADO POR MÉDIA E POR FALTAS':'REPROVADO',
                'REPROVADO POR NOTA E FALTA':'REPROVADO',
                'REPROVADO POR NOTA':'REPROVADO',
                }}, regex=True)'''

"df_dados_filtrado = df_dados_filtrado.replace({'descricao':\n               {'APROVADO POR NOTA':'APROVADO',\n                'REPROVADO POR FALTAS':'REPROVADO',\n                'REPROVADO POR MÉDIA E POR FALTAS':'REPROVADO',\n                'REPROVADO POR NOTA E FALTA':'REPROVADO',\n                'REPROVADO POR NOTA':'REPROVADO',\n                }}, regex=True)"

In [409]:
# Criar uma função para mapear a classificação
def mapear_classificacao(descricao):
    mapeamento = {
        'APROVADO': 2,
        'APROVADO POR NOTA': 1,
        'REPROVADO': -1,
        'REPROVADO POR MÉDIA E POR FALTAS': -2,
        'CANCELADO': 0,
        'EXCLUIDA': 0,
        'TRANCADO': 0,
        'INDEFERIDO': 0,
        'DESISTENCIA': -3,
        'REPROVADO POR FALTAS': -1,
        'REPROVADO POR NOTA': -1,
        'REPROVADO POR NOTA E FALTA': -2,
    }
    return mapeamento.get(descricao, None)

# Aplicar a função de mapeamento e criar a coluna 'classificacao'
df_dados_filtrado.loc[:, 'classificacao'] = df_dados_filtrado['descricao'].map(mapear_classificacao)

# Exibir o DataFrame resultante
print(df_dados_filtrado)


                               discente  unidade  nota media_final  \
0      bad4e0ec61e635f382e7e32c32d5ad11      1.0   0,0         0,0   
1      bad4e0ec61e635f382e7e32c32d5ad11      2.0   0,0         0,0   
2      bad4e0ec61e635f382e7e32c32d5ad11      3.0   0,0         0,0   
3      bee926721a5b23a098fdfb850bf8d5b5      1.0   0,0         0,0   
4      bee926721a5b23a098fdfb850bf8d5b5      2.0   0,0         0,0   
...                                 ...      ...   ...         ...   
54576  22f4aed4a073c5e9515a8669e9c102f3      2.0   6,9         5,2   
54577  22f4aed4a073c5e9515a8669e9c102f3      3.0   5,0         5,2   
54581  943463d2de8c6a60ec5f5d959ba7f1ac      1.0  10,0        10,0   
54582  943463d2de8c6a60ec5f5d959ba7f1ac      2.0  10,0        10,0   
54583  943463d2de8c6a60ec5f5d959ba7f1ac      3.0  10,0        10,0   

       numero_total_faltas             descricao    ano  id_componente  \
0                     44.0  REPROVADO POR FALTAS  20091        2037000   
1          

/tmp/ipykernel_7183/1346963640.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dados_filtrado.loc[:, 'classificacao'] = df_dados_filtrado['descricao'].map(mapear_classificacao)


Contando os valores dos dados da coluna descrição.

In [410]:
df_dados_filtrado.descricao.value_counts()

descricao
APROVADO                            19059
APROVADO POR NOTA                    6933
REPROVADO                            5193
REPROVADO POR MÉDIA E POR FALTAS     3066
CANCELADO                            2032
EXCLUIDA                             1501
TRANCADO                             1360
INDEFERIDO                            402
DESISTENCIA                           216
REPROVADO POR FALTAS                  123
REPROVADO POR NOTA                     81
REPROVADO POR NOTA E FALTA             27
Name: count, dtype: int64

## Extraindo dados

### Colunas com as **Disciplinas**

Calculando a quantidade de vezes que cada discente cursou cada *disciplina*.

In [411]:
quantidade_disciplinas = df_dados_filtrado.groupby(['discente', 'nome']).size().reset_index(name='quantidade')


Pivotando as *disciplinas*.

In [412]:
tabela_final = quantidade_disciplinas.pivot(index='discente', columns='nome', values='quantidade').reset_index()

### Coluna com o **Ano de Ingresso**

Adicionando o *ano_ingresso* para cada discente.

In [413]:
ano_ingresso_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'ano_ingresso']]
tabela_final = tabela_final.merge(ano_ingresso_discente, on='discente', how='left')

### Coluna com o **Sexo** do Discente

Definir a opção global para permitir o downcasting silencioso

In [414]:
pd.set_option('future.no_silent_downcasting', True)

Alterando o conteúdo da coluna *sexo*.

In [415]:
df_dados_filtrado['sexo'] = df_dados_filtrado['sexo'].replace({'M': 0, 'F': 1})
df_dados_filtrado = df_dados_filtrado.infer_objects(copy=False)

/tmp/ipykernel_7183/3799681914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dados_filtrado['sexo'] = df_dados_filtrado['sexo'].replace({'M': 0, 'F': 1})


Adicionando a coluna *sexo*.

In [416]:
sexo_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'sexo']]
tabela_final = tabela_final.merge(sexo_discente, on='discente', how='left')

### Coluna com o **Status** do Discente

Alterando o conteúdo da coluna *status*.

In [417]:
df_dados_filtrado['status'] = df_dados_filtrado['status'].replace({'CANCELADO': -1, 'ATIVO': 0, 'ATIVO - FORMANDO': 0,'FORMADO': 1,'CONCLUÍDO': 1})

Adicionando o *status* para cada discente.

In [418]:
status_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'status']]
tabela_final = tabela_final.merge(status_discente, on='discente', how='left')

In [419]:
# Criar cópia do DataFrame para evitar o aviso SettingWithCopyWarning
df_dados_filtrado = df_dados_filtrado.copy()

# Inicializar a nova coluna com valores padrão
df_dados_filtrado['nota_1'] = pd.NA
df_dados_filtrado['nota_2'] = pd.NA
df_dados_filtrado['nota_3'] = pd.NA

# Preencher as novas colunas com os valores das notas correspondentes à unidade
df_dados_filtrado.loc[df_dados_filtrado['unidade'] == 1, 'nota_1'] = df_dados_filtrado['nota']
df_dados_filtrado.loc[df_dados_filtrado['unidade'] == 2, 'nota_2'] = df_dados_filtrado['nota']
df_dados_filtrado.loc[df_dados_filtrado['unidade'] == 3, 'nota_3'] = df_dados_filtrado['nota']

# Mesclar as colunas de notas na tabela final
tabela_final = tabela_final.merge(df_dados_filtrado[['discente', 'nota_1', 'nota_2', 'nota_3']], on='discente', how='left')
tabela_final

,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,...,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DOS SISTEMAS,ÁLGEBRA LINEAR,ÉTICA,ano_ingresso,sexo,status,nota_1,nota_2,nota_3
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,NaN,NaN,NaN,2018,0,-1,<NA>,<NA>,<NA>
1,001cea3c82e2010681f2cdeab21e5ecf,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,NaN,NaN,NaN,2018,0,-1,<NA>,<NA>,<NA>
2,001cea3c82e2010681f2cdeab21e5ecf,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,NaN,NaN,NaN,2018,0,-1,<NA>,<NA>,<NA>
3,001cea3c82e2010681f2cdeab21e5ecf,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,NaN,NaN,NaN,2018,0,-1,<NA>,<NA>,<NA>
4,001cea3c82e2010681f2cdeab21e5ecf,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,NaN,NaN,NaN,2018,0,-1,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39988,ffe447b2fe2058d45d5f756349a26f45,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,2012,0,-1,<NA>,"0,0",<NA>
39989,ffe447b2fe2058d45d5f756349a26f45,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,2012,0,-1,<NA>,<NA>,"0,0"
39990,ffe447b2fe2058d45d5f756349a26f45,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,2012,0,-1,"0,0",<NA>,<NA>
39991,ffe447b2fe2058d45d5f756349a26f45,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,2012,0,-1,<NA>,"0,0",<NA>


### Coluna com a **Carga Horária Cumprida**

Agrupando por discente e somando a *carga horária*.

In [420]:
df_carga_horaria_cumprida = df_dados_filtrado.groupby('discente')['ch_total'].sum().reset_index()
df_carga_horaria_cumprida.rename(columns={'ch_total': 'ch_cumprida'}, inplace=True)

Adicionando a *carga horária cumprida* de cada discente.

In [421]:
tabela_final = tabela_final.merge(df_carga_horaria_cumprida, on='discente', how='left')

Dividindo a *carga horária cumprida* do discente pela carga horária das disciplinas obrigatórias.

In [422]:
tabela_final['ch_cumprida_dividida'] = tabela_final['ch_cumprida'] / 1830

### Coluna com a **Carga Horária Cumprida** com **Reprovação**

Filtrar as disciplinas *reprovadas*.

In [423]:
df_reprovadas = df_dados_filtrado[df_dados_filtrado['descricao'] == 'REPROVADO']

Agrupar por discente e somar a *carga horária das disciplinas reprovadas*.

In [424]:
ch_reprovacao = df_reprovadas.groupby('discente')['ch_total'].sum().reset_index()

Renomear a coluna resultante.

In [425]:
ch_reprovacao.columns = ['discente', 'ch_reprovacao']

Adicionando a *carga horária de reprovação* de cada discente.

In [426]:
tabela_final = tabela_final.merge(ch_reprovacao, on='discente', how='left')

Substituindo NaN por 0 nas colunas.

In [427]:
tabela_final = tabela_final.fillna(0)

### Coluna com **taxa de reprovação**

In [428]:
tabela_final['taxa_reprovacao'] = tabela_final['ch_reprovacao'] / tabela_final['ch_cumprida'] 
tabela_final

,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,...,ano_ingresso,sexo,status,nota_1,nota_2,nota_3,ch_cumprida,ch_cumprida_dividida,ch_reprovacao,taxa_reprovacao
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2018,0,-1,0,0,0,330,0.180328,0.0,0.0
1,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2018,0,-1,0,0,0,330,0.180328,0.0,0.0
2,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2018,0,-1,0,0,0,330,0.180328,0.0,0.0
3,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2018,0,-1,0,0,0,330,0.180328,0.0,0.0
4,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2018,0,-1,0,0,0,330,0.180328,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39988,ffe447b2fe2058d45d5f756349a26f45,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2012,0,-1,0,"0,0",0,630,0.344262,0.0,0.0
39989,ffe447b2fe2058d45d5f756349a26f45,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2012,0,-1,0,0,"0,0",630,0.344262,0.0,0.0
39990,ffe447b2fe2058d45d5f756349a26f45,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2012,0,-1,"0,0",0,0,630,0.344262,0.0,0.0
39991,ffe447b2fe2058d45d5f756349a26f45,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2012,0,-1,0,"0,0",0,630,0.344262,0.0,0.0


### Coluna com o **Semestre**

Função para calcular o *semestre* com base no *ano de ingresso* e o *ano cursado.*

In [429]:
def calcular_semestre(ano_ingresso, ano_cursado):
    try:
        # Convertendo os anos para strings e ajustando para o formato correto
        ano_ingresso_str = str(ano_ingresso) + '1' if len(str(ano_ingresso)) == 4 else str(ano_ingresso)
        ano_cursado_str = str(ano_cursado) + '1' if len(str(ano_cursado)) == 4 else str(ano_cursado)
        
        ano_ingresso_num = int(ano_ingresso_str[:4])
        semestre_ingresso = int(ano_ingresso_str[4])
        
        ano_cursado_num = int(ano_cursado_str[:4])
        semestre_cursado = int(ano_cursado_str[4])
        
        anos_passados = (ano_cursado_num - ano_ingresso_num) * 2
        semestres_passados = anos_passados + (semestre_cursado - semestre_ingresso)
        
        return semestres_passados + 1  # +1 porque o semestre de ingresso é contado como o primeiro
    except (IndexError, ValueError):
        return pd.NA  # Retorna NA se houver um problema com o formato dos anos

Corrigir os valores das colunas *ano_ingresso* e *ano*.

In [430]:
df_dados_filtrado['ano_ingresso'] = df_dados_filtrado['ano_ingresso'].astype(str)
df_dados_filtrado['ano'] = df_dados_filtrado['ano'].astype(str)

Aplicar a função para criar a coluna *semestre*.

In [431]:
df_dados_filtrado['semestre'] = df_dados_filtrado.apply(
    lambda x: calcular_semestre(x['ano_ingresso'], x['ano']), axis=1
)

In [432]:
###### Adicionar a tabela_final

### Coluna com o Total de **Semestre Cursado**

Identificar *semestres* únicos cursados por cada discente.

In [433]:
semestres_unicos_por_discente = df_dados_filtrado.groupby('discente')['ano'].nunique().reset_index()
semestres_unicos_por_discente.rename(columns={'ano' : 'semestre'}, inplace=True)

Adicionando a quantidade de *semestres* cursados por cada discente.

In [434]:
tabela_final = tabela_final.merge(semestres_unicos_por_discente, on='discente', how='left')

Dividindo a quantidade de *semestres cursados* pela quantidade de *semestres estimados* no curso.

In [435]:
tabela_final['semestre_dividido'] = tabela_final['semestre'] / 8

### Coluna com o **Último Período** Matrículado

Encontrar o *último período* que cada aluno estudou no curso.

In [436]:
ultimo_periodo = df_dados_filtrado.groupby('discente')['ano'].max().reset_index()
ultimo_periodo.rename(columns={'ano': 'ultimo_periodo'}, inplace=True)

Adicionando as informações do *último período* de cada discente.

In [437]:
tabela_final = tabela_final.merge(ultimo_periodo, on='discente', how='left')

In [438]:
# Função para calcular o semestre com base no ano de ingresso e o ano cursado
def calcular_semestre(ano_ingresso, ano_cursado):
    try:
        # Convertendo os anos para strings e ajustando para o formato correto
        ano_ingresso_str = str(ano_ingresso) + '1' if len(str(ano_ingresso)) == 4 else str(ano_ingresso)
        ano_cursado_str = str(ano_cursado) + '1' if len(str(ano_cursado)) == 4 else str(ano_cursado)
        
        ano_ingresso_num = int(ano_ingresso_str[:4])
        semestre_ingresso = int(ano_ingresso_str[4])
        
        ano_cursado_num = int(ano_cursado_str[:4])
        semestre_cursado = int(ano_cursado_str[4])
        
        anos_passados = (ano_cursado_num - ano_ingresso_num) * 2
        semestres_passados = anos_passados + (semestre_cursado - semestre_ingresso)
        
        return semestres_passados + 1  # +1 porque o semestre de ingresso é contado como o primeiro
    except (IndexError, ValueError):
        return pd.NA  # Retorna NA se houver um problema com o formato dos anos

# Corrigir os valores das colunas 'ano_ingresso' e 'ano'
df_dados_filtrado['ano_ingresso'] = df_dados_filtrado['ano_ingresso'].astype(str)
df_dados_filtrado['ano'] = df_dados_filtrado['ano'].astype(str)

# Aplicar a função para criar a coluna 'semestre'
df_dados_filtrado['semestre'] = df_dados_filtrado.apply(
    lambda x: calcular_semestre(x['ano_ingresso'], x['ano']), axis=1
)

### Coluna com a quantidade de vezes que o discente se **Matriculou**

Soma dos valores das colunas para contabilizar a *matrícula* por discente.

In [439]:
matricula =      tabela_final['ALGORITMOS E LÓGICA DE PROGRAMAÇÃO'] + \
                 tabela_final['INTRODUÇÃO À INFORMÁTICA'] + \
                 tabela_final['FUNDAMENTOS DE MATEMÁTICA'] + \
                 tabela_final['LÓGICA'] + \
                 tabela_final['TEORIA GERAL DA ADMINISTRAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO'] + \
                 tabela_final['CÁLCULO DIFERENCIAL E INTEGRAL'] + \
                 tabela_final['TEORIA GERAL DOS SISTEMAS'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS I'] + \
                 tabela_final['ESTRUTURA DE DADOS'] + \
                 tabela_final['ÁLGEBRA LINEAR'] + \
                 tabela_final['ORGANIZAÇÃO, SISTEMAS E MÉTODOS'] + \
                 tabela_final['FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO WEB'] + \
                 tabela_final['ARQUITETURA DE COMPUTADORES'] + \
                 tabela_final['PROBABILIDADE E ESTATÍSTICA'] + \
                 tabela_final['BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE I'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS II'] + \
                 tabela_final['SISTEMAS OPERACIONAIS'] + \
                 tabela_final['PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE II'] + \
                 tabela_final['REDES DE COMPUTADORES'] + \
                 tabela_final['EMPREENDEDORISMO EM INFORMÁTICA'] + \
                 tabela_final['GESTÃO DE PROJETO DE SOFTWARE'] + \
                 tabela_final['PROGRAMAÇÃO VISUAL'] + \
                 tabela_final['MATEMÁTICA FINANCEIRA'] + \
                 tabela_final['SISTEMAS DE APOIO À DECISÃO'] + \
                 tabela_final['ÉTICA']

Adicionando a coluna *matrícula* na tabela.

In [440]:
tabela_final['matricula'] = matricula

### Coluna com o **Tempo Relativo** que o discente passou no curso

Converter a coluna *ch_cumprida_dividida* para tipo numérico.

In [441]:
tabela_final['semestre_dividido'] = pd.to_numeric(tabela_final['semestre_dividido'], errors='coerce')

Converter a coluna *status* para tipo numérico.

In [442]:
tabela_final['status'] = pd.to_numeric(tabela_final['status'], errors='coerce')

Multiplicar os valores da coluna *ch_cumprida_dividida* pela coluna *status*.

In [443]:
tabela_final['tempo_relativo'] = tabela_final['semestre_dividido'] * tabela_final['status']

### Coluna com a quantidade de vezes que o discente foi **Aprovado**

Listar todas as disciplinas.

In [444]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *aprovações*.

In [445]:
aprovacoes = df_dados_filtrado[df_dados_filtrado['descricao'] == 'APROVADO']

Agrupar por discente e nome da disciplina para contar as *aprovações*.

In [446]:
aprovacoes_count = aprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_APROVADO*.

In [447]:
aprovacoes_count.columns = [f"{disciplina}_APROVADO" for disciplina in aprovacoes_count.columns]

Mesclar as *aprovações* de volta ao DataFrame original.

In [448]:
df_final = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(aprovacoes_count, on='discente').fillna(0).reset_index()
tabela_final = tabela_final.merge(df_final, on='discente', how='left')

### Coluna com a quantidade de vezes que o discente foi **Reprovado**

Listar todas as disciplinas.

In [449]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *reprovações*.

In [450]:
reprovacoes = df_dados_filtrado[df_dados_filtrado['descricao'].isin([
    'REPROVADO',
    'REPROVADO POR NOTA',
    'REPROVADO POR FALTAS',
    'REPROVADO POR NOTA E FALTA',
    'REPROVADO POR MÉDIA E POR FALTAS'
])]

Agrupar por discente e nome da disciplina para contar as *reprovações*.

In [451]:
reprovacoes_count = reprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_REPROVADO*.

In [452]:
reprovacoes_count.columns = [f"{disciplina}_REPROVADO" for disciplina in reprovacoes_count.columns]

Mesclar as *reprovações* de volta ao DataFrame original.

In [453]:
df_final_reprovacao = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(reprovacoes_count, on='discente').fillna(0).reset_index()
tabela_final = tabela_final.merge(df_final_reprovacao, on='discente', how='left')

In [454]:
disciplinas = df_dados_filtrado['nome'].unique()

# Filtrar as disciplinas existentes no DataFrame
disciplinas_existentes = [disciplina for disciplina in disciplinas if disciplina in df_dados_filtrado.columns]

# Calcular o total de faltas para cada disciplina por aluno
df_total_faltas = df_dados_filtrado.groupby('discente')[disciplinas_existentes].sum().reset_index()

# Exibir o DataFrame resultante para verificação
print(df_total_faltas)

# Mesclar com o DataFrame original (se necessário)
df_resultado = df_dados_filtrado.merge(df_total_faltas, on='discente', suffixes=('', '_total_faltas'))

df_resultado.to_csv('df_dados_faltas.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

                             discente
0    001cea3c82e2010681f2cdeab21e5ecf
1    005c14d7c07bf7980b60c703f99c5ee7
2    0107fd69d8cd7e3d30dede96fb68bfe5
3    014789363f7940922e71e710ee9d22bc
4    014f0dec46fe7a9c5836527662e1df10
..                                ...
680  fe802d8d85de6f842749468401d1146c
681  fe87dfa176a74fc10a5cb701b9fb5dd4
682  fec9ed6026d55ecdf514c640312c3d08
683  ff56f2c5048dae0797fd3e851572b80c
684  ffe447b2fe2058d45d5f756349a26f45

[685 rows x 1 columns]


In [455]:
# Converter a coluna 'media_final' para tipo numérico se necessário
df_dados_filtrado['media_final'] = pd.to_numeric(df_dados_filtrado['media_final'], errors='coerce')

# Preencher os valores NaN em 'media_final' com 0
df_dados_filtrado['media_final'] = df_dados_filtrado['media_final'].fillna(0)

# Calcular a média final de cada disciplina por discente
media_final_disciplinas = df_dados_filtrado.groupby(['discente', 'nome'])['media_final'].mean().reset_index()

# Pivotar as médias finais
media_final_pivot = media_final_disciplinas.pivot(index='discente', columns='nome', values='media_final').reset_index()

# Renomear as colunas pivotadas para incluir '_MEDIA'
media_final_pivot.columns = [f"{col}_MEDIA" if col != 'discente' else col for col in media_final_pivot.columns]

# Mesclar as médias finais ao DataFrame tabela_final
tabela_final = tabela_final.merge(media_final_pivot, on='discente', how='left')

# Preencher os valores NaN resultantes da pivotagem com 0
tabela_final = tabela_final.fillna(0)

# Listar as colunas que foram adicionadas para confirmar a inclusão
tabela_final.columns

Index(['discente', 'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
       'ARQUITETURA DE COMPUTADORES', 'BANCO DE DADOS',
       'CÁLCULO DIFERENCIAL E INTEGRAL', 'EMPREENDEDORISMO EM INFORMÁTICA',
       'ENGENHARIA DE SOFTWARE I', 'ENGENHARIA DE SOFTWARE II',
       'ESTRUTURA DE DADOS', 'FUNDAMENTOS DE MATEMÁTICA',
       ...
       'PROGRAMAÇÃO VISUAL_MEDIA', 'PROGRAMAÇÃO WEB_MEDIA',
       'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA',
       'REDES DE COMPUTADORES_MEDIA', 'SISTEMAS DE APOIO À DECISÃO_MEDIA',
       'SISTEMAS OPERACIONAIS_MEDIA', 'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA',
       'TEORIA GERAL DOS SISTEMAS_MEDIA', 'ÁLGEBRA LINEAR_MEDIA',
       'ÉTICA_MEDIA'],
      dtype='object', length=168)

In [456]:
tabela_final.to_csv('tabelateste.csv', index=False, sep=';', quoting=csv.QUOTE_NONNUMERIC)

Mudando a ordem das colunas.

In [ ]:
'''colunas_ordenadas = ['discente', 'sexo', 'ano_ingresso', 'ultimo_periodo', 'ch_cumprida',
                    'ch_cumprida_dividida', 'ch_reprovacao',
                    'taxa_reprovacao', 'semestre', 'semestre_dividido', 'tempo_relativo', 'status', 'matricula',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FREQUENCIA',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'INTRODUÇÃO À INFORMÁTICA',
                    'INTRODUÇÃO À INFORMÁTICA_APROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_REPROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_FREQUENCIA',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_1',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_2',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_3',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_1',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_2',
                    'FUNDAMENTOS DE MATEMÁTICA',
                    'FUNDAMENTOS DE MATEMÁTICA_APROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'LÓGICA',
                    'LÓGICA_APROVADO',
                    'LÓGICA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'TEORIA GERAL DA ADMINISTRAÇÃO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_APROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_FREQUENCIA',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_1',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_2',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_3',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_1',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_2',
                    'PROGRAMAÇÃO',
                    'PROGRAMAÇÃO_APROVADO',
                    'PROGRAMAÇÃO_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'CÁLCULO DIFERENCIAL E INTEGRAL',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_APROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'TEORIA GERAL DOS SISTEMAS',
                    'TEORIA GERAL DOS SISTEMAS_APROVADO',
                    'TEORIA GERAL DOS SISTEMAS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ESTRUTURA DE DADOS',
                    'ESTRUTURA DE DADOS_APROVADO',
                    'ESTRUTURA DE DADOS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ÁLGEBRA LINEAR',
                    'ÁLGEBRA LINEAR_APROVADO',
                    'ÁLGEBRA LINEAR_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_APROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_APROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'PROGRAMAÇÃO WEB',
                    'PROGRAMAÇÃO WEB_APROVADO',
                    'PROGRAMAÇÃO WEB_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ARQUITETURA DE COMPUTADORES',
                    'ARQUITETURA DE COMPUTADORES_APROVADO',
                    'ARQUITETURA DE COMPUTADORES_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'PROBABILIDADE E ESTATÍSTICA',
                    'PROBABILIDADE E ESTATÍSTICA_APROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'BANCO DE DADOS',
                    'BANCO DE DADOS_APROVADO',
                    'BANCO DE DADOS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ENGENHARIA DE SOFTWARE I',
                    'ENGENHARIA DE SOFTWARE I_APROVADO',
                    'ENGENHARIA DE SOFTWARE I_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'SISTEMAS OPERACIONAIS',
                    'SISTEMAS OPERACIONAIS_APROVADO',
                    'SISTEMAS OPERACIONAIS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_APROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ENGENHARIA DE SOFTWARE II',
                    'ENGENHARIA DE SOFTWARE II_APROVADO',
                    'ENGENHARIA DE SOFTWARE II_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'REDES DE COMPUTADORES',
                    'REDES DE COMPUTADORES_APROVADO',
                    'REDES DE COMPUTADORES_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'EMPREENDEDORISMO EM INFORMÁTICA',
                    'EMPREENDEDORISMO EM INFORMÁTICA_APROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'GESTÃO DE PROJETO DE SOFTWARE',
                    'GESTÃO DE PROJETO DE SOFTWARE_APROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'PROGRAMAÇÃO VISUAL',
                    'PROGRAMAÇÃO VISUAL_APROVADO',
                    'PROGRAMAÇÃO VISUAL_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'MATEMÁTICA FINANCEIRA',
                    'MATEMÁTICA FINANCEIRA_APROVADO',
                    'MATEMÁTICA FINANCEIRA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'SISTEMAS DE APOIO À DECISÃO',
                    'SISTEMAS DE APOIO À DECISÃO_APROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ÉTICA',
                    'ÉTICA_APROVADO',
                    'ÉTICA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2']
'''

"colunas_ordenadas = ['discente', 'sexo', 'ano_ingresso', 'ultimo_periodo', 'ch_cumprida',\n                    'ch_cumprida_dividida', 'ch_reprovacao',\n                    'taxa_reprovacao', 'semestre', 'semestre_dividido', 'tempo_relativo', 'status', 'matricula',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FREQUENCIA',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2',\n                    'INTRODUÇÃO À INFORMÁTICA',\n                    'INTRODUÇÃO À INFORMÁTICA_APROVADO',\n               

Reordenando as colunas.

In [ ]:
tabela_final = tabela_final[colunas_ordenadas]
tabela_final

NameError: name 'colunas_ordenadas' is not defined

Para salvar o DataFrame no formato CSV com a separação por ';' e garantir que os dados numéricos estejam no formato correto.

In [ ]:
tabela_final.to_csv('tabela_final.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)